## Dataset Preparation

In [ ]:
import kagglehub
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score

### Preparing the dataframe

In [ ]:
path = kagglehub.dataset_download("vikramamin/bank-loan-approval-lr-dt-rf-and-auc")
csv = os.path.join(path, "bankloan.csv")

Using Colab cache for faster access to the 'bank-loan-approval-lr-dt-rf-and-auc' dataset.


In [ ]:
df = pd.read_csv(csv)
df

,ID,Age,Experience,Income,ZIP.Code,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


### Removing unnecessary columns

In [ ]:
df = df.drop(columns=["ID", "ZIP.Code"])
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,0,1,0
4996,30,4,15,4,0.4,1,85,0,0,0,1,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,0,1,0


### Converting negative values in the `Experience` column to absolute values

In [ ]:
df["Experience"] = abs(df["Experience"])
df

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,0,0,0,1,0
4996,30,4,15,4,0.4,1,85,0,0,0,1,0
4997,63,39,24,2,0.3,3,0,0,0,0,0,0
4998,65,40,49,3,0.5,2,0,0,0,0,1,0


### Checking for null values

In [ ]:
df.isnull().sum()

,0
Age,0
Experience,0
Income,0
Family,0
CCAvg,0
Education,0
Mortgage,0
Personal.Loan,0
Securities.Account,0
CD.Account,0


### Checking for categorical columns

In [ ]:
categorical_columns = df.select_dtypes(include=["object", "category", "string", "bool"]).columns.tolist()
categorical_columns

[]

### Converting all columns to numeric types

In [ ]:
df = df.apply(pd.to_numeric, errors="coerce")
df.dtypes

,0
Age,int64
Experience,int64
Income,int64
Family,int64
CCAvg,float64
Education,int64
Mortgage,int64
Personal.Loan,int64
Securities.Account,int64
CD.Account,int64


### Checking the `df`'s shape, info, and statistical summary

In [ ]:
df.shape

(5000, 12)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 5000 non-null   int64  
 1   Experience          5000 non-null   int64  
 2   Income              5000 non-null   int64  
 3   Family              5000 non-null   int64  
 4   CCAvg               5000 non-null   float64
 5   Education           5000 non-null   int64  
 6   Mortgage            5000 non-null   int64  
 7   Personal.Loan       5000 non-null   int64  
 8   Securities.Account  5000 non-null   int64  
 9   CD.Account          5000 non-null   int64  
 10  Online              5000 non-null   int64  
 11  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 468.9 KB


In [ ]:
df.describe()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal.Loan,Securities.Account,CD.Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,45.338400,20.134600,73.774200,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,11.463166,11.415189,46.033729,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,23.000000,0.000000,8.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,35.000000,10.000000,39.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,45.000000,20.000000,64.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,55.000000,30.000000,98.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,67.000000,43.000000,224.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


### Normalizing the data

In [ ]:
X = df.drop(columns=["Personal.Loan"]).values
y = df["Personal.Loan"].values
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled[0]

array([0.04545455, 0.02325581, 0.18981481, 1.        , 0.16      ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        ])

## Base Neural Network

### Creating custom PyTorch dataset class

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = torch.tensor(x_data, dtype=torch.float32)
    self.y_data = torch.tensor(y_data, dtype=torch.float32).unsqueeze(1)

  def __getitem__(self, idx):
    return (self.x_data[idx], self.y_data[idx])

  def __len__(self):
    return len(self.x_data)

### Creating Base Neural Network

In [ ]:
class BaseNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()

    self.input_to_h1 = nn.Linear(11, 9)
    self.h1_to_h2 = nn.Linear(9, 6)
    self.h2_to_h3 = nn.Linear(6, 3)
    self.h3_to_output = nn.Linear(3, 1)

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.input_to_h1.forward(x)
    x = self.sigmoid.forward(x)

    x = self.h1_to_h2.forward(x)
    x = self.sigmoid.forward(x)

    x = self.h2_to_h3.forward(x)
    x = self.sigmoid.forward(x)

    x = self.h3_to_output.forward(x)

    return x

### Preparing the dataloaders

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.3, random_state=42, shuffle=True)
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Training the model with baseline architecture

In [ ]:
model = BaseNeuralNetwork()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.BCEWithLogitsLoss()

In [ ]:
def train_fn(model, optimizer, loss_fn, train_dataloader, val_dataloader):
  ave_loss = 0.0
  val_loss = 0.0

  # ave_loss
  model.train()
  for x, y in train_dataloader:
    predictions = model.forward(x)

    loss = loss_fn(predictions, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ave_loss += loss.item()

  ave_loss = ave_loss / len(train_dataloader)

  # val_loss
  model.eval()

  for x, y in val_dataloader:
    predictions = model.forward(x)
    loss = loss_fn(predictions, y)

    val_loss += loss.item()

  val_loss = val_loss / len(val_dataloader)

  return ave_loss, val_loss

In [ ]:
epochs = 100

for i in range(epochs):
  ave_loss, val_loss = train_fn(model, optimizer, loss_function, train_loader, val_loader)

  print(f"Epoch {i+1}: Ave Loss: {ave_loss} Val Loss: {val_loss}")

Epoch 1: Ave Loss: 0.4652408762411638 Val Loss: 0.43435034663119215
Epoch 2: Ave Loss: 0.39383694502440364 Val Loss: 0.3845079078319225
Epoch 3: Ave Loss: 0.3534186382185329 Val Loss: 0.35912370301307517
Epoch 4: Ave Loss: 0.3322282177480784 Val Loss: 0.3465516129706768
Epoch 5: Ave Loss: 0.32095361785455184 Val Loss: 0.34034371344333
Epoch 6: Ave Loss: 0.31483315175229853 Val Loss: 0.3373911773904841
Epoch 7: Ave Loss: 0.31149330938404257 Val Loss: 0.3361147333015787
Epoch 8: Ave Loss: 0.30968063954602587 Val Loss: 0.3356787329341503
Epoch 9: Ave Loss: 0.30870737717910246 Val Loss: 0.3356357663869858
Epoch 10: Ave Loss: 0.3081896049055186 Val Loss: 0.3357488917226487
Epoch 11: Ave Loss: 0.3079116775230928 Val Loss: 0.33589567703769563
Epoch 12: Ave Loss: 0.3077512129463933 Val Loss: 0.33601283741758226
Epoch 13: Ave Loss: 0.30763418125835335 Val Loss: 0.336059643867168
Epoch 14: Ave Loss: 0.3075017851184715 Val Loss: 0.33598123300582805
Epoch 15: Ave Loss: 0.30726876854896545 Val Loss

In [ ]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for xb, yb in val_loader:
        outputs = model(xb)
        preds = torch.round(torch.sigmoid(outputs))
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(yb.cpu().numpy().flatten())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

conf_matrix = confusion_matrix(all_labels, all_preds)
TN, FP, FN, TP = conf_matrix.ravel()

accuracy = accuracy_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
specificity = TN / (TN + FP)

print("Accuracy:", accuracy)
print("Recall (Sensitivity):", recall)
print("Precision:", precision)
print("F1-score:", f1)
print("Specificity:", specificity)

Accuracy: 0.974
Recall (Sensitivity): 0.8598726114649682
Precision: 0.8881578947368421
F1-score: 0.8737864077669902
Specificity: 0.9873417721518988


## Proposed Network Modification

Sources:
- [SERF Activation Function](https://openaccess.thecvf.com/content/WACV2023/papers/Nag_SERF_Towards_Better_Training_of_Deep_Neural_Networks_Using_Log-Softplus_WACV_2023_paper.pdf)  
- [Skip Connections (ResNet)](https://arxiv.org/pdf/1701.09175.pdf)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
import torch.nn.functional as F

### Custom Activation Function

In [ ]:
class SERF(tf.keras.layers.Layer):
    def __init__(self):
        super(SERF, self).__init__()

    def call(self, inputs):
        return inputs * tf.math.erf(tf.math.log(1 + tf.exp(inputs)))

### Neural Network with Skip Connection

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x1 = tf.keras.layers.Dense(9)(inputs)
x1 = SERF()(x1)

x2 = tf.keras.layers.Dense(9)(x1)
x2 = SERF()(x2)
res1 = tf.keras.layers.Add()([x2, x1])

x3 = tf.keras.layers.Dense(6)(res1)
x3 = SERF()(x3)

x4 = tf.keras.layers.Dense(3)(x3)
x4 = SERF()(x4)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x4)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 9)         │        108 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ serf (SERF)         │ (None, 9)         │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 9)         │         90 │ serf[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ serf_1 (SERF)       │ (None, 9)         │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 9)         │          0 │ serf_1[0][0],     │
│                     │                   │            │ serf[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 6)         │         60 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ serf_2 (SERF)       │ (None, 6)         │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │         21 │ serf_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ serf_3 (SERF)       │ (None, 3)         │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │          4 │ serf_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 283 (1.11 KB)

 Trainable params: 283 (1.11 KB)

 Non-trainable params: 0 (0.00 B)

### Training modified model

In [ ]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.4597 - loss: 0.6953 - val_accuracy: 0.8953 - val_loss: 0.4121
Epoch 2/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8939 - loss: 0.3771 - val_accuracy: 0.8953 - val_loss: 0.3375
Epoch 3/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9003 - loss: 0.3255 - val_accuracy: 0.8953 - val_loss: 0.3292
Epoch 4/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8974 - loss: 0.3232 - val_accuracy: 0.8953 - val_loss: 0.3206
Epoch 5/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9114 - loss: 0.2851 - val_accuracy: 0.8953 - val_loss: 0.3005
Epoch 6/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8967 - loss: 0.2876 - val_accuracy: 0.8953 - val_loss: 0.2472
Epoch 7/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9113 - loss: 0.2186 - val_accuracy: 0.9253 - val_loss: 0.1848
Epoch 8/100
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9338 - loss: 0.1783 - val_acc

In [ ]:
predictions = model.predict(X_val)
predictions = (predictions > 0.5).astype(int)

conf_matrix = confusion_matrix(predictions, y_val)
TN, FP, FN, TP = conf_matrix.ravel()

accuracy = accuracy_score(predictions, y_val)
recall = recall_score(predictions, y_val)
precision = precision_score(predictions, y_val)
f1 = f1_score(predictions, y_val)
specificity = TN / (TN + FP)

print("Accuracy:", accuracy)
print("Recall (Sensitivity):", recall)
print("Precision:", precision)
print("F1-score:", f1)
print("Specificity:", specificity)

47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Accuracy: 0.9866666666666667
Recall (Sensitivity): 0.9790209790209791
Precision: 0.89171974522293
F1-score: 0.9333333333333333
Specificity: 0.9874723655121592
